In [5]:
import pandas as pd
from datetime import datetime, timedelta
pd.set_option('display.max_rows', 500) 
pd.options.display.float_format = '{:20,.8f}'.format

input_dt = datetime.today()
first = input_dt.replace(day=1)
last_day = first - timedelta(days=1)
date_yyyymmdd = last_day.strftime('%Y%m%d')
date_yymm = last_day.strftime('%y%m')
date_yyyy = last_day.strftime('%Y')

df = pd.read_csv(r'\\10.186.237.122\grcm\GCRM-GEN\GCCRM\Group Corporate Credit Risk\DSP Utilisation\Monthly\MAN_MR_MY_EQ.M' + date_yyyymmdd + '.csv')
Template_DSP = pd.read_excel(r'\\10.186.237.122\grcm\GCRM-GEN\GCCRM\Group Corporate Credit Risk\DSP Utilisation\Template DSP.xlsx', sheet_name='working')

Template_DSP.sort_values("New_ISIN", inplace=True)
Template_DSP['New_ISIN'] = Template_DSP['New_ISIN'].str.replace(" ", "")
Template_DSP.drop_duplicates(subset="New_ISIN", inplace=True, keep="first")

df['UNIT_MARKET_VALUE'] = df['UNIT_MARKET_VALUE'].astype('float64')
df['ISIN'] = df['ISIN'].str.replace(" ", "")
df['Utilisation_Value_RM'] = df['HOLDING_UNIT'] * df['UNIT_MARKET_VALUE']
df2 = df[['ISIN','Utilisation_Value_RM']].groupby(['ISIN'], as_index=False).agg(Utilisation_Value_RM = ('Utilisation_Value_RM' ,'sum')).query("Utilisation_Value_RM != 0").sort_values(['Utilisation_Value_RM'], ascending=False)
merged = (pd.merge(df2,Template_DSP, how='left',  left_on ='ISIN' , right_on='New_ISIN'))[['Bloomberg name','ISIN', 'Utilisation_Value_RM']]


merged.rename(columns = {'Bloomberg name':'NAME'}, inplace = True)
merged['CategoryName'] = 'DSP_Equity'
merged = merged[['NAME','ISIN', 'CategoryName','Utilisation_Value_RM']]



writer = pd.ExcelWriter(r'\\10.186.237.122\grcm\GCRM-GEN\GCCRM\Group Corporate Credit Risk\DSP Utilisation\\' + date_yyyy +'\DSP Utilisation_'+ date_yymm + '_python.xlsx', engine='xlsxwriter')

#  Convert the dataframe to an XlsxWriter Excel object.
merged.to_excel(writer, sheet_name='Sheet1',index = False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

# Set a currency number format for a column.
num_format = workbook.add_format({'num_format': '#,###.00'})
worksheet.set_column('D:D', None, num_format)

# Close the Pandas Excel writer and output the Excel file.

writer.close()


